
<h3>
Python 3 scripts that does the following<br>

 Create a startup script to install Apache server and copy the website files from S3 to the
instance.<br>
 Create a Python program using boto that creates an Amazon AutoScaling group. Define
scale up and scale down policies and corresponding CloudWatch alarms. Supply the
startup script you created in previous step while launching a new instance from the
program. Use a security group port 80 open.<br>
 Open the public DNS of the newly launched instance in a browser and verify if the static
website works.<br>
</h3>

First we import boto3 and create an instance of the client class for AutoScaling <br>

Then we write our user script for hosting Apache server and create a new launch configuration for creating <br>
our AWS AutoScaling Group and pass the necessary parameters as given in the documentation 
<a href="https://boto3.readthedocs.io/en/latest/reference/services/autoscaling.html">here</a><br>


In [1]:
import boto3

from pprint import pprint

as_client = boto3.client('autoscaling')

user_data_script ="""#!/bin/bash
sudo apt-get update
sudo apt-get install -y build-essential
sudo apt-get install -y apache2
curl https://s3.ap-south-1.amazonaws.com/www.kaushik.das/index.html > index.html
sudo rm /var/www/html/index.html
sudo mv index.html /var/www/html/index.html"""


response_lc = as_client.create_launch_configuration(
                
                LaunchConfigurationName='MyAutoScalingLaunchConfig',
                ImageId='ami-04169656fea786776',
                KeyName='ec2-1',
                SecurityGroups=['sg-0fffb748eaf870000'],
                UserData=user_data_script,
                InstanceType='t2.micro',
                InstanceMonitoring={
                    'Enabled':True
                },
                AssociatePublicIpAddress=True,
                    
                
            )





AlreadyExistsFault: An error occurred (AlreadyExists) when calling the CreateLaunchConfiguration operation: Launch Configuration by this name already exists - A launch configuration already exists with the name MyAutoScalingLaunchConfig

After we have our launch configuration we create our autoscaling group. 

In [2]:
response_asg = as_client.create_auto_scaling_group(

                AutoScalingGroupName = 'MyAutoScalingGroup',
                LaunchConfigurationName = 'MyAutoScalingLaunchConfig',
                MinSize = 1,
                MaxSize = 3,
                DesiredCapacity = 1,
                HealthCheckType = 'EC2',
                VPCZoneIdentifier = 'subnet-3715906b'
    
            )


In [3]:
pprint(response_asg)

{'ResponseMetadata': {'HTTPHeaders': {'content-length': '231',
                                      'content-type': 'text/xml',
                                      'date': 'Wed, 29 Aug 2018 03:51:38 GMT',
                                      'x-amzn-requestid': 'd530d805-ab3e-11e8-840c-790e82f5c0ab'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'd530d805-ab3e-11e8-840c-790e82f5c0ab',
                      'RetryAttempts': 0}}


We print the details about the autoscaling group


In [4]:
r = as_client.describe_auto_scaling_groups(
    AutoScalingGroupNames=[
        'MyAutoScalingGroup',
    ],
    MaxRecords=10
)

pprint(r)

{'AutoScalingGroups': [{'AutoScalingGroupARN': 'arn:aws:autoscaling:us-east-1:117198229476:autoScalingGroup:a7040423-9afb-4e7a-bcbb-ebec8f507da8:autoScalingGroupName/MyAutoScalingGroup',
                        'AutoScalingGroupName': 'MyAutoScalingGroup',
                        'AvailabilityZones': ['us-east-1c'],
                        'CreatedTime': datetime.datetime(2018, 8, 29, 3, 51, 39, 107000, tzinfo=tzutc()),
                        'DefaultCooldown': 300,
                        'DesiredCapacity': 1,
                        'EnabledMetrics': [],
                        'HealthCheckGracePeriod': 0,
                        'HealthCheckType': 'EC2',
                        'Instances': [{'AvailabilityZone': 'us-east-1c',
                                       'HealthStatus': 'Healthy',
                                       'InstanceId': 'i-0a6cb2db56a699897',
                                       'LaunchConfigurationName': 'MyAutoScalingLaunchConfig',
                       

Next we create a scale up policy for our autoscaling group.<br>
The policy below allows the number of instances to be increased by one.




In [5]:
response_asgp = as_client.put_scaling_policy(

                    AutoScalingGroupName = 'MyAutoScalingGroup',
                    PolicyName = 'ASGPolicy',
                    PolicyType = 'SimpleScaling',
                    AdjustmentType = 'ChangeInCapacity',
                    ScalingAdjustment = -1,
    
                )

We parse the response and save the PolicyARN in a variable for later use with CloudWatch Alarm

In [6]:
pprint(response_asgp)
action_down = response_asgp.get('PolicyARN')
print(action_down)

{'Alarms': [],
 'PolicyARN': 'arn:aws:autoscaling:us-east-1:117198229476:scalingPolicy:314eb676-7702-49aa-ac57-fe1186d0d2ca:autoScalingGroupName/MyAutoScalingGroup:policyName/ASGPolicy',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '470',
                                      'content-type': 'text/xml',
                                      'date': 'Wed, 29 Aug 2018 03:53:34 GMT',
                                      'x-amzn-requestid': '1a0a6bb4-ab3f-11e8-b40c-75bb3b5d6933'},
                      'HTTPStatusCode': 200,
                      'RequestId': '1a0a6bb4-ab3f-11e8-b40c-75bb3b5d6933',
                      'RetryAttempts': 0}}
arn:aws:autoscaling:us-east-1:117198229476:scalingPolicy:314eb676-7702-49aa-ac57-fe1186d0d2ca:autoScalingGroupName/MyAutoScalingGroup:policyName/ASGPolicy


Similarly , we create a scale down policy that allows for the number of instances to be decreased by one.<br>
We save the PoliyARN for later use with cloudWatch.

In [7]:
response_asgpu = as_client.put_scaling_policy(

                    AutoScalingGroupName = 'MyAutoScalingGroup',
                    PolicyName = 'ASGPolicyUp',
                    PolicyType = 'SimpleScaling',
                    AdjustmentType = 'ChangeInCapacity',
                    ScalingAdjustment = +1,
    
                )

In [8]:
pprint(response_asgpu)
action_up = response_asgpu.get('PolicyARN')
print(action_up)

{'Alarms': [],
 'PolicyARN': 'arn:aws:autoscaling:us-east-1:117198229476:scalingPolicy:e427f228-5c26-44b7-a633-21b2b5582f11:autoScalingGroupName/MyAutoScalingGroup:policyName/ASGPolicyUp',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '472',
                                      'content-type': 'text/xml',
                                      'date': 'Wed, 29 Aug 2018 03:53:56 GMT',
                                      'x-amzn-requestid': '27883ffb-ab3f-11e8-97b4-0b64b5f1862e'},
                      'HTTPStatusCode': 200,
                      'RequestId': '27883ffb-ab3f-11e8-97b4-0b64b5f1862e',
                      'RetryAttempts': 0}}
arn:aws:autoscaling:us-east-1:117198229476:scalingPolicy:e427f228-5c26-44b7-a633-21b2b5582f11:autoScalingGroupName/MyAutoScalingGroup:policyName/ASGPolicyUp


Next ,we set up a <a href="https://boto3.readthedocs.io/en/latest/reference/services/cloudwatch.html">CloudWatch alarm</a>, the particular alarm below keeps checking the autoscaling group instances for two<br> points of threshhold breach every two minutes. The threshhold in the alarm below is described by CPU <br>Utilization. When the cpu utilization raises above 70, it takes action as per our scale up policy and creates<br> another instance. <br>

Some metrics for the threshold and their detailed explaination can be found in<a href="https://docs.aws.amazon.com/AmazonCloudWatch/latest/monitoring/ec2-metricscollected.html
"> here</a>.<br>


In [9]:
cloudwatch_client = boto3.client('cloudwatch')

response_alarm_up = cloudwatch_client.put_metric_alarm(

                AlarmName = 'MyASGAlarm_scaleUp',
                AlarmDescription = 'AutoScaling Group Instance Alarm',
                ActionsEnabled = True,
                
                AlarmActions = [
                        action_up
                ],
    
                MetricName = 'CPUUtilization',
                Namespace = 'AWS/EC2',
                Statistic = 'Average',
                Dimensions = [
                    {
                    'Name': 'AutoScalingGroupName',
                    'Value': 'MyAutoScalingGroup'
                    }
                ],
                Period = 60,
                Unit = 'Percent',
                EvaluationPeriods = 2,
                Threshold = 70,
                ComparisonOperator= 'GreaterThanThreshold'
    

            )


# GreaterThanThreshold, LessThanThreshold, LessThanOrEqualToThreshold, GreaterThanOrEqualToThreshold]



In [10]:
pprint(response_alarm_up)

{'ResponseMetadata': {'HTTPHeaders': {'content-length': '214',
                                      'content-type': 'text/xml',
                                      'date': 'Wed, 29 Aug 2018 03:55:10 GMT',
                                      'x-amzn-requestid': '53ad0f3c-ab3f-11e8-a0cd-013671332683'},
                      'HTTPStatusCode': 200,
                      'RequestId': '53ad0f3c-ab3f-11e8-a0cd-013671332683',
                      'RetryAttempts': 0}}


Similar thing here, we create another alarm that monitors CPU utilization, and calls upon the scale down policy incase the CPU utiliztion decreases below 40%.

In [11]:

response_alarm_down = cloudwatch_client.put_metric_alarm(

                AlarmName = 'MyASGAlarm_scaleDown',
                AlarmDescription = 'AutoScaling Group Instance Alarm',
                ActionsEnabled = True,
                
                AlarmActions = [
                        action_down
                ],
    
                MetricName = 'CPUUtilization',
                Namespace = 'AWS/EC2',
                Statistic = 'Average',
                Dimensions = [
                    {
                    'Name': 'AutoScalingGroupName',
                    'Value': 'MyAutoScalingGroup'
                    }
                ],
                Period = 60,
                Unit = 'Percent',
                EvaluationPeriods = 2,
                Threshold = 40,
                ComparisonOperator= 'LessThanThreshold'
    

            )


In [12]:
pprint(response_alarm_down)

{'ResponseMetadata': {'HTTPHeaders': {'content-length': '214',
                                      'content-type': 'text/xml',
                                      'date': 'Wed, 29 Aug 2018 03:55:28 GMT',
                                      'x-amzn-requestid': '5e289fe7-ab3f-11e8-b191-2f8851e635e0'},
                      'HTTPStatusCode': 200,
                      'RequestId': '5e289fe7-ab3f-11e8-b191-2f8851e635e0',
                      'RetryAttempts': 0}}


Now we delete our autoScaling group to free resources

In [ ]:
response_del = as_client.delete_auto_scaling_group(
    AutoScalingGroupName='MyAutoScalingGroup',
    ForceDelete=True
)